# Install Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install pyspark==2.4.5

In [3]:
!ls

hmp.parquet		       spark-3.0.0-bin-hadoop2.7.tgz.1
sample_data		       spark-3.0.0-bin-hadoop2.7.tgz.2
spark-3.0.0-bin-hadoop2.7      spark-3.0.0-bin-hadoop2.7.tgz.3
spark-3.0.0-bin-hadoop2.7.tgz


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [6]:
try:
    from pyspark import SparkContext, SparkConf
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [7]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

## Exercise

In [8]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-07-22 05:54:29--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-07-22 05:54:29--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-07-22 05:54:29--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (91

In [9]:
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [10]:
df_two_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")

In [11]:
df_two_class.show()

+---+---+---+--------------------+-------------+
|  x|  y|  z|              source|        class|
+---+---+---+--------------------+-------------+
| 30| 40| 51|Accelerometer-201...|Standup_chair|
| 30| 41| 51|Accelerometer-201...|Standup_chair|
| 31| 41| 51|Accelerometer-201...|Standup_chair|
| 29| 42| 51|Accelerometer-201...|Standup_chair|
| 30| 43| 52|Accelerometer-201...|Standup_chair|
| 30| 40| 52|Accelerometer-201...|Standup_chair|
| 31| 41| 52|Accelerometer-201...|Standup_chair|
| 32| 39| 52|Accelerometer-201...|Standup_chair|
| 29| 38| 52|Accelerometer-201...|Standup_chair|
| 29| 38| 50|Accelerometer-201...|Standup_chair|
| 28| 40| 50|Accelerometer-201...|Standup_chair|
| 31| 38| 51|Accelerometer-201...|Standup_chair|
| 30| 39| 51|Accelerometer-201...|Standup_chair|
| 30| 39| 50|Accelerometer-201...|Standup_chair|
| 31| 39| 51|Accelerometer-201...|Standup_chair|
| 30| 38| 52|Accelerometer-201...|Standup_chair|
| 29| 39| 53|Accelerometer-201...|Standup_chair|
| 31| 38| 52|Acceler

In [12]:
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [13]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [15]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="label", featuresCol="features_norm", maxIter=10)

In [16]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, gbt])

In [17]:
model = pipeline.fit(df_train)

In [18]:
prediction = model.transform(df_train)

In [19]:
prediction.show()

+---+---+---+--------------------+-------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  x|  y|  z|              source|        class|label|       features|       features_norm|       rawPrediction|         probability|prediction|
+---+---+---+--------------------+-------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  0| 31| 17|Accelerometer-201...|Standup_chair|  0.0|[0.0,31.0,17.0]|[0.0,0.4423076923...|[1.32190952082575...|[0.93362900698360...|       0.0|
|  0| 31| 32|Accelerometer-201...|Standup_chair|  0.0|[0.0,31.0,32.0]|[0.0,0.4423076923...|[1.32190952082575...|[0.93362900698360...|       0.0|
|  0| 31| 32|Accelerometer-201...|Standup_chair|  0.0|[0.0,31.0,32.0]|[0.0,0.4423076923...|[1.32190952082575...|[0.93362900698360...|       0.0|
|  0| 32| 33|Accelerometer-201...|Standup_chair|  0.0|[0.0,32.0,33.0]|[0.0,0.4615384615...|[1.32190952082575...|[0.93362900698360.

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.9111746090269457

In [21]:
prediction = model.transform(df_test)

In [22]:
binEval.evaluate(prediction) 

0.9080914144533663

In [24]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(normalizer.p, [1.0, 2.0, 10.0]) \
    .addGrid(gbt.maxBins, [2,4,8,16]) \
    .addGrid(gbt.maxDepth, [2,4,8,16]) \
    .build()

In [25]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=4)  

In [26]:
cvModel = crossval.fit(df_train)

In [27]:
prediction = cvModel.transform(df_test)

In [28]:
binEval.evaluate(prediction) 

0.9109326744904261

In [29]:
cvModel.bestModel.stages[-1].explainParams()

"cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)\nfeatureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)